In [1]:
import torch
from torch import nn, optim
import torchvision
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style("whitegrid"); sns.set_palette("tab10")
import pandas as pd
import numpy as np
from torch.autograd import Variable
from models.vae import VAE

In [ ]:
data = 